In [2]:
import pandas as pd

In [3]:
dalys_haq_sdi_answers_out_significant_ageStd = \
pd.read_csv('/ihme/homes/arjuns13/notebooks/Documents/Data/dalys_haq_sdi_allcauses_ageStd_10Jun19.csv')

In [4]:
dalys_haq_sdi_answers_out_significant_ageStd.shape

(167, 14)

In [5]:
dalys_haq_sdi_answers_out_significant_ageStd['cause_id'] = dalys_haq_sdi_answers_out_significant_ageStd['cause_id'].astype('int64')

In [6]:
dalys_haq_sdi_answers_out_significant_ageStd['sex_id'] = dalys_haq_sdi_answers_out_significant_ageStd['sex_id'].astype('int64')

In [7]:
dalys_haq_sdi_answers_out_significant_ageStd['age_group_id'] = dalys_haq_sdi_answers_out_significant_ageStd['age_group_id'].astype('int64')

In [8]:
dalys_haq_sdi_answers_out_significant_ageStd.head()

,Unnamed: 0,cause_id,age_group_id,sex_id,intercept,intercept_pValue,haq_over_frontier_coef,haq_over_frontier_coef_pValue,logit_sdi_coef,logit_sdi_coef_pValue,r2,age_group_name,cause_name,sex
0,2,298,27,1,0.027086,2.281757e-51,-6.358639e-05,1.715718e-03,-0.006637,1.513699e-02,0.153045,Age-standardized,HIV/AIDS,Male
1,3,298,27,2,0.027011,6.439200e-50,-4.595813e-05,2.524703e-02,-0.009890,3.689814e-04,0.164460,Age-standardized,HIV/AIDS,Female
2,4,302,27,1,0.018031,6.496932e-174,1.685661e-05,4.871939e-03,-0.013610,3.422434e-55,0.560209,Age-standardized,Diarrheal diseases,Male
3,6,321,27,1,0.000016,5.048977e-146,4.101624e-08,3.102767e-11,-0.000016,1.694059e-67,0.523738,Age-standardized,Other intestinal infectious diseases,Male
4,7,321,27,2,0.000020,4.841675e-133,6.351323e-08,7.178372e-15,-0.000022,9.135697e-73,0.522734,Age-standardized,Other intestinal infectious diseases,Female


In [9]:
dalys_haq_sdi_answers_out_significant_ageStd = dalys_haq_sdi_answers_out_significant_ageStd[['cause_name', 'cause_id','age_group_name',\
                                                                              'age_group_id', 'sex', 'sex_id', \
                                                                              'haq_over_frontier_coef']]

In [10]:
dalys_haq_sdi_answers_out_significant_ageStd.head()

,cause_name,cause_id,age_group_name,age_group_id,sex,sex_id,haq_over_frontier_coef
0,HIV/AIDS,298,Age-standardized,27,Male,1,-6.358639e-05
1,HIV/AIDS,298,Age-standardized,27,Female,2,-4.595813e-05
2,Diarrheal diseases,302,Age-standardized,27,Male,1,1.685661e-05
3,Other intestinal infectious diseases,321,Age-standardized,27,Male,1,4.101624e-08
4,Other intestinal infectious diseases,321,Age-standardized,27,Female,2,6.351323e-08


In [11]:
dalys_haq_sdi_answers_out_significant_ageStd.shape

(167, 7)

## Get the data for the HAQ-over-frontier values

In [12]:
haqFront_sdi_daly_data = pd.read_csv('/ihme/homes/arjuns13/notebooks/Documents/Data/haq_sdi_frontier_estimates_ageStd_aggregatedCauses_allLocsIncUS.csv')

In [13]:
haqFront_sdi_daly_data = haqFront_sdi_daly_data[haqFront_sdi_daly_data['location_id']==102]

In [14]:
haqFront_sdi_daly_data.head()

,Unnamed: 0,V1,location_id,logit_sdi,age_group_id,ln_haq,fit2
823,824,0,102,1.86831,27,4.374564,4.693256


In [15]:
haqFront_sdi_daly_data['age_group_id'].unique()

array([27])

In [16]:
final = haqFront_sdi_daly_data

In [17]:
final['haq'] = 10**(final['ln_haq'])

In [18]:
final['exp_fit'] = 10**(final['fit2'])

In [19]:
final.loc[final['exp_fit'] > 100.0, 'exp_fit']=100


In [20]:
final['exp_fit'].describe()

count      1.0
mean     100.0
std        NaN
min      100.0
25%      100.0
50%      100.0
75%      100.0
max      100.0
Name: exp_fit, dtype: float64

In [21]:
final['haq_over_frontier'] = final['haq']/final['exp_fit']

In [22]:
final[final['haq_over_frontier']==1].age_group_id.unique() ## make sure that HAQ over frontier = 1 for age group IDs > 19

array([], dtype=int64)

## End of the copied part from the previously done analysis

In [23]:
final.head()

,Unnamed: 0,V1,location_id,logit_sdi,age_group_id,ln_haq,fit2,haq,exp_fit,haq_over_frontier
823,824,0,102,1.86831,27,4.374564,4.693256,23689.957588,100.0,236.899576


In [24]:
final.shape

(1, 10)

## Getting SDI back from Logit SDI using the Expit function

In [25]:
from scipy import special
final['sdi'] = special.expit(final['logit_sdi'])

## To be merged (on Location Id and Age Group ID) with the DALY data coming from the central funcs - here I end up keeping LogitSDI, rather than SDI, which is what I used for the 2nd June run

In [26]:
haq_sdi = final[['age_group_id', 'haq_over_frontier']]

In [27]:
haq_sdi.head()

,age_group_id,haq_over_frontier
823,27,236.899576


In [28]:
haq_sdi.shape

(1, 2)

## Merge HAQ-over-frontier values with the "dalys_haq_sdi_answers_out_significant_withoutOld" dataset

In [29]:
dalys_haqFront_sdi = pd.merge(dalys_haq_sdi_answers_out_significant_ageStd, haq_sdi, on=['age_group_id'])

In [30]:
dalys_haqFront_sdi.shape

(167, 8)

In [31]:
dalys_haqFront_sdi.head()

,cause_name,cause_id,age_group_name,age_group_id,sex,sex_id,haq_over_frontier_coef,haq_over_frontier
0,HIV/AIDS,298,Age-standardized,27,Male,1,-6.358639e-05,236.899576
1,HIV/AIDS,298,Age-standardized,27,Female,2,-4.595813e-05,236.899576
2,Diarrheal diseases,302,Age-standardized,27,Male,1,1.685661e-05,236.899576
3,Other intestinal infectious diseases,321,Age-standardized,27,Male,1,4.101624e-08,236.899576
4,Other intestinal infectious diseases,321,Age-standardized,27,Female,2,6.351323e-08,236.899576


## Getting the population:

### To keep things consistent with the age-group-specific regressions/predictions we use 22 (all ages) - {21 (80+) + 20 (75-79)} 

In [83]:
from db_queries import get_population

US_pop = get_population(age_group_id=[20,21,22], location_id=102, year_id=2016, sex_id=[1,2], gbd_round_id=5)

In [84]:
US_pop = US_pop[['age_group_id', 'sex_id', 'population']]

In [85]:
US_pop

,age_group_id,sex_id,population
0,20,1,3.646713e+06
1,21,1,4.567506e+06
2,22,1,1.589647e+08
3,20,2,4.529333e+06
4,21,2,7.351801e+06
5,22,2,1.639074e+08


In [86]:
US_pop_piv = pd.pivot_table(US_pop,\
                                    values='population', index=['sex_id'],\
                                    columns=['age_group_id'], fill_value='NA')

In [87]:
US_pop_piv

age_group_id,20,21,22
sex_id,,,
1,3.646713e+06,4.567506e+06,1.589647e+08
2,4.529333e+06,7.351801e+06,1.639074e+08


In [114]:
 US_pop_piv.iloc[0]

age_group_id
20    3.646713e+06
21    4.567506e+06
22    1.589647e+08
Name: 1, dtype: float64

In [115]:
US_pop_piv_un = pd.DataFrame(columns=['pop', 'sex_id'])

In [118]:
for i in range(len(US_pop_piv)):
    a = US_pop_piv.iloc[i].iloc[2]-US_pop_piv.iloc[i].iloc[0]-US_pop_piv.iloc[i].iloc[1]
    b = US_pop_piv.iloc[i].name
    US_pop_piv_un.loc[i] = [a,b]

In [119]:
US_pop_piv_un

,pop,sex_id
0,1.507505e+08,1.0
1,1.520263e+08,2.0


In [123]:
dalys_haqFront_sdi_withPop = pd.merge(dalys_haqFront_sdi, US_pop_piv_un, on=['sex_id'] )

In [124]:
dalys_haqFront_sdi_withPop.shape

(167, 9)

In [125]:
dalys_haqFront_sdi_withPop.head()

,cause_name,cause_id,age_group_name,age_group_id,sex,sex_id,haq_over_frontier_coef,haq_over_frontier,pop
0,HIV/AIDS,298,Age-standardized,27,Male,1,-6.358639e-05,236.899576,1.507505e+08
1,Diarrheal diseases,302,Age-standardized,27,Male,1,1.685661e-05,236.899576,1.507505e+08
2,Other intestinal infectious diseases,321,Age-standardized,27,Male,1,4.101624e-08,236.899576,1.507505e+08
3,Lower respiratory infections,322,Age-standardized,27,Male,1,-1.966858e-05,236.899576,1.507505e+08
4,Upper respiratory infections,328,Age-standardized,27,Male,1,1.345372e-06,236.899576,1.507505e+08


In [126]:
dalys_haqFront_sdi_withPop['Predicted_DALYs'] = \
dalys_haqFront_sdi_withPop['haq_over_frontier_coef'] * dalys_haqFront_sdi_withPop['haq_over_frontier'] * dalys_haqFront_sdi_withPop['pop']

In [127]:
dalys_haqFront_sdi_withPop.head()

,cause_name,cause_id,age_group_name,age_group_id,sex,sex_id,haq_over_frontier_coef,haq_over_frontier,pop,Predicted_DALYs
0,HIV/AIDS,298,Age-standardized,27,Male,1,-6.358639e-05,236.899576,1.507505e+08,-2.270843e+06
1,Diarrheal diseases,302,Age-standardized,27,Male,1,1.685661e-05,236.899576,1.507505e+08,6.019954e+05
2,Other intestinal infectious diseases,321,Age-standardized,27,Male,1,4.101624e-08,236.899576,1.507505e+08,1.464802e+03
3,Lower respiratory infections,322,Age-standardized,27,Male,1,-1.966858e-05,236.899576,1.507505e+08,-7.024184e+05
4,Upper respiratory infections,328,Age-standardized,27,Male,1,1.345372e-06,236.899576,1.507505e+08,4.804690e+04


In [128]:
dalys_haqFront_sdi_withPop_final = dalys_haqFront_sdi_withPop[['cause_name', 'cause_id', 'sex_id', 'Predicted_DALYs']]

In [129]:
dalys_haqFront_sdi_withPop_final.head()

,cause_name,cause_id,sex_id,Predicted_DALYs
0,HIV/AIDS,298,1,-2.270843e+06
1,Diarrheal diseases,302,1,6.019954e+05
2,Other intestinal infectious diseases,321,1,1.464802e+03
3,Lower respiratory infections,322,1,-7.024184e+05
4,Upper respiratory infections,328,1,4.804690e+04


In [130]:
dalys_haqFront_sdi_withPop_final.shape

(167, 4)

## Now onto the pivotting part

## Segregating for men and women

In [131]:
dalys_haqFront_sdi_withPop_final_male = \
dalys_haqFront_sdi_withPop_final[dalys_haqFront_sdi_withPop_final['sex_id']==1]

In [132]:
dalys_haqFront_sdi_withPop_final_male = dalys_haqFront_sdi_withPop_final_male.drop(columns=['sex_id'])

In [133]:
dalys_haqFront_sdi_withPop_final_male.head()

,cause_name,cause_id,Predicted_DALYs
0,HIV/AIDS,298,-2.270843e+06
1,Diarrheal diseases,302,6.019954e+05
2,Other intestinal infectious diseases,321,1.464802e+03
3,Lower respiratory infections,322,-7.024184e+05
4,Upper respiratory infections,328,4.804690e+04


In [134]:
dalys_haqFront_sdi_withPop_final_male.shape

(87, 3)

In [135]:
dalys_haqFront_sdi_withPop_final_female = \
dalys_haqFront_sdi_withPop_final[dalys_haqFront_sdi_withPop_final['sex_id']==2]

In [136]:
dalys_haqFront_sdi_withPop_final_female = dalys_haqFront_sdi_withPop_final_female.drop(columns=['sex_id'])

In [137]:
dalys_haqFront_sdi_withPop_final_female.head()

,cause_name,cause_id,Predicted_DALYs
87,HIV/AIDS,298,-1.655180e+06
88,Other intestinal infectious diseases,321,2.287426e+03
89,Upper respiratory infections,328,6.128763e+04
90,Meningitis,332,2.522087e+05
91,Diphtheria,338,4.968978e+03


In [138]:
dalys_haqFront_sdi_withPop_final_female.shape

(80, 3)

In [139]:
dalys_haqFront_sdi_withPop_final_female.to_csv('/ihme/homes/arjuns13/notebooks/Documents/Data/dalys_haq_sdi_females_ageStd_PredictedDALYs_10Jun19.csv')

In [140]:
dalys_haqFront_sdi_withPop_final_female.to_csv('J:\\Project\\Cost_Effectiveness\\NPC\\Regression_Analysis\\dalys_haq_sdi_females_ageStd_PredictedDALYs_10Jun19.csv')

In [141]:
dalys_haqFront_sdi_withPop_final_male.to_csv('/ihme/homes/arjuns13/notebooks/Documents/Data/dalys_haq_sdi_males_ageStd_PredictedDALYs_10Jun19.csv')

In [142]:
dalys_haqFront_sdi_withPop_final_male.to_csv('J:\\Project\\Cost_Effectiveness\\NPC\\Regression_Analysis\\dalys_haq_sdi_males_ageStd_PredictedDALYs_10Jun19.csv')